In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os, sys

# Add the 'modules' folder to the import path
sys.path.append(os.path.join('..', 'modules'))

# Now you can import modules from the 'modules' folder
import utils, acv, models  
cf = utils.load_yaml_file(os.path.join('..', 'config', 'env.yaml'))


In [5]:
# laoding functions from utils.py
import pandas as pd
pd.set_option('display.max_columns', None)
from pathlib import Path
basepath=Path(cf['data_path'])
#from utils import load_initial_files,filter_records,create_Features,preprocess,modelling
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

# Merging Input Files

In [6]:
#You only Need to run this one time, initial_merge= 0 after intial run and saving Final_merge
initial_merge=0
if initial_merge:
    df=utils.load_initial_files(basepath, cf['data_files'], True, 'final_merge.csv')
    

# Filtering and preprocessing

In [7]:
filter_data=0

if filter_data: 
    df=pd.read_csv(basepath/'final_merge.csv')
    df=acv.filter_records(df,500,None,None,None,True,basepath/'final_500_none_none_none_true.csv',True)
    print("Filtering Data :done")
    


In [8]:
create_features=1
if create_features: 
    df=pd.read_csv(basepath/'final_500_none_none_none_true.csv')
    df=acv.create_features(df)
    print("Creating Features :done")
df.head()

............
Creating Features :done


,auction_bid_count,auction_buyer_gross_purchased_amount_to_date,auction_buyer_net_purchased_amount_to_date,auction_distance_lister_to_high_bidder,auction_distinct_bidding_dealers_count,auction_distinct_bidding_users_count,auction_distinct_viewing_users_count,distinct_accepted_offer_count,auction_has_accepted_offer,auction_has_offer,auction_accepted_offer_amount,distinct_offer_count,distinct_offer_made_users_count,distinct_offer_made_dealers_count,auction_end_time,auction_is_no_reserve,auction_is_bid_sale,auction_end_time_legacy,true_auction_end_time,auction_end_time_at_launch,auction_active_time,auction_expired_time,auction_ending_amount,auction_floor_price,auction_gross_run_number,auction_high_bid,auction_second_high_bid_for_dealmade,auction_high_bidder_id,auction_high_bid_is_proxy,auction_id,auction_is_dealmade,auction_is_dealmaker_touched,dealmaker_touched_admin_notes,dealmaker_touched_admin_notes_updated_time,auction_is_green_light,red_light_reason,auction_red_light_changed_to_green_in_past,auction_is_gross_sold,auction_is_live_appraisal,lister_dealer_gross_sold_vehicle_vin_as_live_appraisal_in_past,vehicle_vin_gross_sold_as_live_appraisal_in_past,live_appriasal_marketing_event,auction_is_net_sold,auction_is_run_list,auction_was_preview_lane,auction_is_private,auction_private_lane,auction_is_unwound,auction_is_yellow_light,yellow_light_reason,auction_yellow_light_was_removed_in_past,auction_is_blue_light,auction_lister_id,auction_odometer,auction_original_vci_user_id,auction_pickup_city,auction_pickup_state,auction_pickup_territory,auction_pickup_region,auction_pickup_area,auction_pickup_territory_id,auction_pickup_zip,saved_auction_id,auction_launch_time,sa_creator_vci_id,inspection_guid,inspection_listed_as_private,inspection_first_completion_time,inspection_was_unsold_private_auction,check_engine_light_selected,check_engine_light_notes,auction_sale_amount,auction_start_price,auction_created_time,auction_start_time,auction_status,auction_status_expired,auction_vehicle_id,auction_view_count,auction_is_won_by_programmatic_bid,price_band_base,buyer_total,seller_total,capital_fee_name_at_checkout,capital_base_fee_amount_at_checkout,capital_extension_flat_fee_amount_at_checkout,capital_extension_daily_fee_amount_at_checkout,vehicle_superset,vehicle_segment,lister_dealer_account_manager_id,lister_dealer_country,lister_dealer_is_active,lister_dealer_is_preferred_seller,lister_dealer_is_live_appraisal_seller,lister_dealer_live_appraisal_seller_last_updated_time,lister_dealer_is_live_appraisal_service_fee_eligible,lister_dealer_territory,lister_dealer_area,lister_dealer_territory_id,lister_dealer_territory_manager_id,lister_dealer_commercial_sub_type,lister_dealer_group_is_top_150,lister_dealer_is_acv_auctions_customer,lister_dealer_is_asi_dealership,lister_dealer_is_max_digital_account,lister_dealer_max_digital_business_unit_id,lister_dealer_is_drivably_account,lister_dealer_is_true_frame_dealership,lister_dealer_is_acv_anywhere_dealership,lister_dealer_acv_anywhere_registraion_date,lister_payment_providers_enabled,lister_dealer_was_capital_accelerate_approved,lister_dealer_was_netsuite_approved_capital_dealer,lister_dealer_preferred_buyer_payment_method,lister_user_id,auction_launched_by_user_id,auction_launched_by_user_is_staff,auction_launched_device_platform,lister_am_id,lister_tm_id,lister_pc_id,lister_terr_region,lister_terr_area,lister_terr_territory,high_bidder_dealer_account_manager_id,high_bidder_dealer_country,high_bidder_dealer_is_preferred_seller,high_bidder_dealer_is_preferred_buyer,high_bidder_dealer_territory,high_bidder_dealer_area,high_bidder_dealer_territory_id,high_bidder_territory_manager_id,high_bidder_dealer_group_id,high_bidder_dealer_group_is_top_150,high_bidder_is_buying_matrix_user,high_bidder_dealer_is_acv_auctions_customer,high_bidder_dealer_is_asi_dealership,high_bidder_dealer_is_max_digital_account,high_bidder_dealer_max_digital_business_unit_id,high_bidde_dealer_is_drivably_account,high_bidder_dea

# Preprocessing 

In [9]:
# varibles defining
misc=cf['misc']
cat_controls= cf['cat_controls']
num_controls = cf['num_controls']
dvs=cf['dvs']

utils.are_sets_non_overlapping(cat_controls, num_controls, dvs) 

allcols=key+cat_controls+num_controls+dvs
df=df[allcols]
df.isna().sum()


auction_id                                         0
vehicle_vin                                        0
auction_is_green_light                             0
auction_is_yellow_light                            0
auction_is_blue_light                              0
auction_red_light_changed_to_green_in_past         0
is_holiday                                         0
auction_hour                                       0
auction_dow                                        0
auction_end_month                                  0
auction_year                                       0
lister_dealer_territory                            0
auction_gross_run_number_calc                      0
auction_lister_id                                  0
week_num                                           0
auction_odometer                                   0
auction_week                                       0
auction_is_gross_sold                              0
auction_sale_amount                           

# Model Selection

#  Auction Floor price

In [10]:
run_reg=0

import time
start_time = time.time()
results= pd.DataFrame()
regmodels=models.get_regmodels()

if run_reg==1:
    for dv in ['auction_floor_price','auction_sale_amount','auction_ending_amount','auction_bid_count','auction_distinct_bidding_users_count','auction_distinct_viewing_users_count']:
        
        for rmodel in regmodels:
            print(dv, rmodel,  time.time()-start_time)
            ex={'experiment': dv,'df':df, 'training_weeks':55, 'cat': cat_controls, 'num':num_controls, 'dv': dv, 'key':key, 'type':'regression','model': rmodel }
            if dv=='auction_sale_amount':
                ex['df']=df[df['auction_is_gross_sold']==1]
            temp_results=models.train_test_split_validation(ex)
            #print('r2', temp_results['r2'][1])
            results=pd.concat([results, temp_results])
            results.to_csv(basepath/'mlresults_v2.csv', index=False)
    results

In [14]:
run_class=1
import time
start_time = time.time()
results_cat= pd.DataFrame()
classmodels=models.get_classmodels()
if run_class==1:
    for dv in ['auction_is_gross_sold']:
        for cmodel in classmodels:
            print(dv, cmodel,  time.time()-start_time)
            ex={'experiment': dv,'df':df, 'training_weeks':55, 'cat': cat_controls, 'num':num_controls, 'dv': dv, 'key':key, 'type':'classification','model': cmodel }
            temp_results=models.train_test_split_validation(ex)
            #print('r2', temp_results['r2'][1])
            results_cat=pd.concat([results_cat, temp_results])
            results_cat.to_csv(basepath/'mlresults_cat.csv', index=False)
    results_cat

auction_is_gross_sold LogsReg 0.0
auction_is_gross_sold CatBoostClassifier 17.883689403533936
auction_is_gross_sold GradBoostClass 97.69038200378418
auction_is_gross_sold XGB 336.33120346069336


In [13]:
results_cat

,sysdate,sample_size,experiment,dv,model,train_test,recall,accuracy,balanced_acc,precision,confusion_matrix,log_loss,roc_auc_score,brier_score_loss
0,2023-07-27 22:27:47.849804,794512,auction_is_gross_sold,auction_is_gross_sold,XGB,train,0.830375,0.727251,0.726293,0.691247,"[[244902, 148697], [68005, 332908]]",0.549678,0.796106,0.184488
0,2023-07-27 22:27:49.001382,604552,auction_is_gross_sold,auction_is_gross_sold,XGB,test,0.784645,0.683304,0.69517,0.603796,"[[207367, 134995], [56464, 205726]]",0.588567,0.750899,0.201767


In [ ]:
#TODO: ADD REGRESSION OUTCOME OF # of AUCTIONs
     #: ADD CAT OUTCOME OF SELL THROUGH 

#For this you would include same variables just only the first auction for each vehcile. 